## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('data/train.csv')

### Data Preprocessing

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df=df.dropna()

In [5]:
X=df.drop('label',axis=1)

In [6]:
y=df['label']

In [7]:
X.shape

(18285, 4)

In [8]:
y.shape

(18285,)

In [9]:
import tensorflow as tf

In [10]:
tf.__version__

'2.9.1'

In [24]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import *
from tensorflow.keras.layers import Dense

In [12]:
voc_size=5000

### Onehot Representation

In [13]:
messages=X.copy()

In [14]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [15]:
messages.reset_index(inplace=True)

In [16]:
import nltk
import re
from nltk.corpus import stopwords

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Sai
[nltk_data]     Kalyan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [19]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [20]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[4662, 3287, 1230, 2225, 554, 3767, 2037, 3492, 3493, 4272],
 [46, 3958, 1382, 3452, 4848, 2700, 3263],
 [4956, 4480, 3054, 4896],
 [2, 364, 1531, 3532, 3000, 2343],
 [2105, 4848, 4476, 3426, 1574, 2768, 4848, 4998, 8, 3983],
 [812,
  30,
  943,
  929,
  3722,
  4206,
  1099,
  4622,
  2470,
  3390,
  4629,
  4593,
  3548,
  4449,
  3263],
 [2155, 1447, 2835, 4419, 2737, 2452, 1810, 4280, 929, 200, 113],
 [1335, 2007, 3927, 878, 3526, 4528, 4206, 834, 929, 200, 113],
 [342, 4991, 1897, 1589, 365, 1261, 1245, 3581, 4206, 3531],
 [1900, 523, 669, 2832, 841, 1757, 148, 79],
 [474, 3940, 942, 2539, 3013, 607, 2536, 2574, 1746, 2554, 3829],
 [3532, 4825, 554, 1261, 4206, 3526],
 [32, 4606, 3960, 2668, 4686, 3852, 269, 3963, 3786],
 [4853, 1203, 912, 4342, 1471, 33, 4052, 929, 200, 113],
 [893, 3046, 3948, 1764, 2914, 929, 200, 113],
 [285, 134, 4248, 3105, 2690, 721, 3975, 4251, 1587, 3840],
 [1134, 2025, 3958],
 [4605, 3223, 4529, 4088, 4206, 2631, 873, 3263],
 [836, 3648, 1382, 4507, 274

### Embedding Representation

In [21]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3492 3493 4272]
 [   0    0    0 ... 4848 2700 3263]
 [   0    0    0 ... 4480 3054 4896]
 ...
 [   0    0    0 ...  929  200  113]
 [   0    0    0 ... 2359  681 4344]
 [   0    0    0 ... 1883  277  215]]


In [22]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4662,
       3287, 1230, 2225,  554, 3767, 2037, 3492, 3493, 4272])

### Building The Model

In [70]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_4 (LSTM)               (None, 20, 50)            18200     
                                                                 
 dropout_9 (Dropout)         (None, 20, 50)            0         
                                                                 
 lstm_5 (LSTM)               (None, 50)                20200     
                                                                 
 dropout_10 (Dropout)        (None, 50)                0         
                                                                 
 dense_4 (Dense)             (None, 1)                 51        
                                                                 
Total params: 238,451
Trainable params: 238,451
Non-tr

In [71]:
len(embedded_docs),y.shape

(18285, (18285,))

In [72]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [73]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

### Train Test Split

In [74]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [75]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
383/383 [==============================] - 29s 44ms/step - loss: 0.2831 - accuracy: 0.8660 - val_loss: 0.1920 - val_accuracy: 0.9188
Epoch 2/10
383/383 [==============================] - 17s 44ms/step - loss: 0.1322 - accuracy: 0.9496 - val_loss: 0.2048 - val_accuracy: 0.9133
Epoch 3/10
383/383 [==============================] - 14s 37ms/step - loss: 0.0912 - accuracy: 0.9678 - val_loss: 0.2406 - val_accuracy: 0.9140
Epoch 4/10
383/383 [==============================] - 15s 39ms/step - loss: 0.0558 - accuracy: 0.9815 - val_loss: 0.3077 - val_accuracy: 0.9138
Epoch 5/10
383/383 [==============================] - 14s 37ms/step - loss: 0.0377 - accuracy: 0.9877 - val_loss: 0.3253 - val_accuracy: 0.9067
Epoch 6/10
383/383 [==============================] - 14s 38ms/step - loss: 0.0237 - accuracy: 0.9926 - val_loss: 0.3785 - val_accuracy: 0.9105
Epoch 7/10
383/383 [==============================] - 16s 41ms/step - loss: 0.0163 - accuracy: 0.9953 - val_loss: 0.4473 - val_accuracy:

### Performance Metrics And Accuracy

In [63]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")

189/189 [==============================] - 1s 2ms/step


In [64]:
y_pred

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [65]:
from sklearn.metrics import confusion_matrix

In [66]:
confusion_matrix(y_test,y_pred)

array([[3118,  301],
       [ 244, 2372]], dtype=int64)

In [67]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9096934548467275

In [68]:
from sklearn.metrics import f1_score
y_pred_class = y_pred > 0.5
f1= f1_score(y_test, y_pred_class)

In [69]:
f1

0.896955946303649